In [1]:
# dataset
!gdown 1mk8unxmoCq1Yo77k-6W0g4z8qBvDrL54

Downloading...
From: https://drive.google.com/uc?id=1mk8unxmoCq1Yo77k-6W0g4z8qBvDrL54
To: /content/dataset_builders.xlsx
100% 681k/681k [00:00<00:00, 190MB/s]


In [2]:
!nvidia-smi

Sun Jun 25 05:33:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 22.1 MB/s eta 0:00:00


In [4]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 89.4 MB/s eta 0:00:00


In [5]:
!pip install rouge_score
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=45c745cb33fbb234a564275267020cf4ba72f9bb63c6848a13f1a704f483d700
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 10.4 MB/s eta 0:00:00


In [6]:
!pip install git+https://github.com/huggingface/transformers.git
# !pip install -U sentence-transformers
!pip install evaluate
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-d29h41lt
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-d29h41lt
  Resolved https://github.com/huggingface/transformers.git to commit 8e164c5400b7b413c7b8fb32e35132001effc970
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7206883 sha256=7aafb84cd92f8eb19c0f2c83f4bce841585f3c510c5a86fac7867e608ba545e9
  Stored in directory: /tmp

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Установка библиотек

In [8]:
# Conda
# conda install protobuf=3.20

# Conda/Colab
# pip install sentence_transformers

In [9]:
import re
# from datetime import datetime

import pandas as pd
import numpy as np

# from simpletransformers.question_answering import QuestionAnsweringModel

# import torch
# from torch.utils.data import DataLoader

# from sentence_transformers import SentenceTransformer, losses, InputExample, evaluation, models, util

from sklearn.model_selection import train_test_split

In [10]:
import os
import re
import json

import numpy as np
import pandas as pd

import evaluate
from datasets import load_dataset, Dataset

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from sklearn.model_selection import train_test_split
import torch

In [11]:
os.environ["WANDB_DISABLED"] = "true"

In [12]:
data = pd.read_excel('dataset_builders.xlsx')

data.head(5)

,id,name(название),specialization(специализация),responsibilities(Должностные обязанности),requirements(Требования к соискателю),terms(Условия),skills(Ключевые навыки),salary_from,salary_to,object,...,updated_by,position,phone,website,email,image,unique_code,city_code,source_id,link_resource
0,9495846,Арматурщик,Арматурщик,Работа на строительных площадках. Очистных соо...,NaN,NaN,NaN,150000,170000,NaN,...,\N,Арматурщик,NaN,hh,hh@hh.ru,\N,HH-81058502,\N,1,https://hh.ru/vacancy/81058502
1,9495850,Арматурщик,Арматурщик,Гнутье арматурной стали на механических станка...,NaN,NaN,NaN,130000,150000,NaN,...,\N,Арматурщик,NaN,hh,hh@hh.ru,\N,HH-80163818,\N,1,https://hh.ru/vacancy/80163818
2,9495851,Арматурщик,Арматурщик,Вязка арматуры.,NaN,NaN,NaN,85000,90000,NaN,...,\N,Арматурщик,NaN,hh,hh@hh.ru,\N,HH-77729491,\N,1,https://hh.ru/vacancy/77729491
3,2,Арматурщик,Арматурщик,Выполнение работ по гнутью и резке арматурной ...,Выполнение работ по гнутью и резке арматурной ...,NaN,\N,50000,100000,"ООО ""АЛЬМИС_ИНТЕГРАЛ""",...,1,Арматурщик,74959214241,https://almisintegral.ru/,info@almisintegral.ru,\N,\N,\N,1,\N
4,9388857,Арматурщик,Арматурщик Вахта в мcк (питание+проживание) 60/30,Вахта в город Москва. Обязанности: - армирова...,опыт в строительстве приветствуется; работа в ...,продолжительность вахты 60/30 (продление вахты...,NaN,67000,134000,NaN,...,\N,Арматурщик,NaN,avito,avito@avito.ru,\N,A-3037298578,\N,1,https://avito.ru/3037298578


In [13]:
data = data[['id', 'name(название)', 'specialization(специализация)', 'responsibilities(Должностные обязанности)',
                                     'requirements(Требования к соискателю)', 'terms(Условия)']]

data.columns = ["id", "name", "spec", "resp", "reqr", "terms"]

In [14]:
data[data == r"\N"] = float('nan')

data = data.fillna("")

data = data.astype(str)

In [15]:
def make_full_text(df):
    return f"{df['name']} | {df['spec']}\n{df['resp']}"

data["full_text"] = data.apply(lambda x: make_full_text(x), axis=1)

data.head(5)

,id,name,spec,resp,reqr,terms,full_text
0,9495846,Арматурщик,Арматурщик,Работа на строительных площадках. Очистных соо...,,,Арматурщик | Арматурщик\nРабота на строительны...
1,9495850,Арматурщик,Арматурщик,Гнутье арматурной стали на механических станка...,,,Арматурщик | Арматурщик\nГнутье арматурной ста...
2,9495851,Арматурщик,Арматурщик,Вязка арматуры.,,,Арматурщик | Арматурщик\nВязка арматуры.
3,2,Арматурщик,Арматурщик,Выполнение работ по гнутью и резке арматурной ...,Выполнение работ по гнутью и резке арматурной ...,,Арматурщик | Арматурщик\nВыполнение работ по г...
4,9388857,Арматурщик,Арматурщик Вахта в мcк (питание+проживание) 60/30,Вахта в город Москва. Обязанности: - армирова...,опыт в строительстве приветствуется; работа в ...,продолжительность вахты 60/30 (продление вахты...,Арматурщик | Арматурщик Вахта в мcк (питание+п...


In [16]:
data4t5 = []

for _, row in data.iterrows():
    data4t5.append({'input': 'Извлеки информацию: ' + row['full_text'],
                    'target': 'ТРЕБОВАНИЯ К СОИСКАТЕЛЮ: ' + row['reqr'] + '\n' +
                              'УСЛОВИЯ: ' + row['terms']})

# data4t5

In [17]:
train_data, val_data = train_test_split(
    # data4t5[['id', 'full_text', 'reqr', 'terms']],
    data4t5,
    random_state=43, test_size=.1
)

# train_data

In [18]:
train_data = Dataset.from_list(train_data)
# val_data = Dataset.from_list(val_data)

In [19]:
train_data[0]

{'input': 'Извлеки информацию: Инженер | Инженер\nРабота со сметной и проектной документацией, организация закупок строительных материалов на объект, организация строительного процесса, составление отчетной документации, взаимодействие с заказчиком, ведение исполнительной документации. ГСМ и сотовая связь оплачивается.',
 'target': 'ТРЕБОВАНИЯ К СОИСКАТЕЛЮ: \nУСЛОВИЯ: ГСМ и сотовая связь оплачивается.'}

In [20]:
CKPT = 'ai-forever/ruT5-large'
tokenizer = AutoTokenizer.from_pretrained(CKPT)
model = AutoModelForSeq2SeqLM.from_pretrained(CKPT)

In [21]:
# tokenize the examples
MAX_LENGHT_INPUT = 512
MAX_LENGHT_TARGET = 256
def convert_to_features(example_batch):
    input_encodings = tokenizer(
        example_batch['input'], pad_to_max_length=True, max_length=MAX_LENGHT_INPUT)

    target_encodings = tokenizer(
        example_batch['target'], pad_to_max_length=True, max_length=MAX_LENGHT_TARGET)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [22]:
train_data = train_data.map(convert_to_features, batched=True, remove_columns=train_data.column_names)
# val_data = val_data.map(convert_to_features, batched=True, remove_columns=val_data.column_names)

columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']

train_data.set_format(type='torch', columns=columns)
# val_data.set_format(type='torch', columns=columns)

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2398: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [24]:
rouge = evaluate.load("rouge")
bleu = evaluate.load("sacrebleu")
accuracy = evaluate.load("accuracy")
wer = evaluate.load("wer")

In [25]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    # print(decoded_preds, '\n', decoded_labels)

    bleu_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    accuracy_result = sum([
        decoded_preds[i] in decoded_labels[i] for i in range(len(decoded_preds))
                                                        ]) / len(decoded_preds)


    return {
        "accuracy" :  accuracy_result,
        "bleu":       bleu_result["score"],
        "rouge1":     rouge_result["rouge1"],
        "rouge2":     rouge_result["rouge2"],
        "rougeL":     rouge_result["rougeL"],
        "rougeLsum" : rouge_result["rougeLsum"],
        }


def compute_metrics(preds, targets):
    bleu_result = bleu.compute(predictions=preds, references=targets)
    rouge_result = rouge.compute(predictions=preds, references=targets)
    wer_result = wer.compute(predictions=preds, references=targets)

    return {
        "bleu":       bleu_result["score"],
        "rouge1":     rouge_result["rouge1"],
        "rouge2":     rouge_result["rouge2"],
        "rougeL":     rouge_result["rougeL"],
        "rougeLsum" : rouge_result["rougeLsum"],
        "wer" :       wer_result,
        }

In [32]:
# set training arguments - Feel free to adapt it

### TODO data_collator

training_args = Seq2SeqTrainingArguments(
    output_dir="models/rut5-base-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    # per_device_eval_batch_size=32,
    predict_with_generate=True,
    # evaluation_strategy="epoch",
    # eval_steps=1000,

    # do_train=True,
    # do_eval=True,
    logging_steps=500,
    save_strategy="epoch",
    #save_steps=1000,
    overwrite_output_dir=True,
    save_total_limit=3,
    # load_best_model_at_end=True,
    # push_to_hub=True,
    # fp16=True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [33]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    # compute_metrics=compute_metrics,
    train_dataset=train_data,
    # eval_dataset=val_data,
)

In [ ]:
trainer.train()

In [34]:
trainer.save_model()

In [35]:
tokenizer.save_pretrained("models/rut5-base-finetuned")

('models/rut5-base-finetuned/tokenizer_config.json',
 'models/rut5-base-finetuned/special_tokens_map.json',
 'models/rut5-base-finetuned/spiece.model',
 'models/rut5-base-finetuned/added_tokens.json',
 'models/rut5-base-finetuned/tokenizer.json')

In [40]:
# path = "/content/models/rut5-base-finetuned-ru-spider/checkpoint-450"

# model = AutoModelForSeq2SeqLM.from_pretrained(path)
# tokenizer = AutoTokenizer.from_pretrained(path)

In [36]:
from tqdm.auto import tqdm

In [ ]:
predsictions = []
truths = []

device = "cuda"

for item in tqdm(val_data):
    with torch.no_grad()
        input_ids = tokenizer.encode(item["input"], return_tensors="pt", add_special_tokens=True).to(device)
        generated_ids = model.generate(input_ids=input_ids, max_length=512, repetition_penalty=2.5, early_stopping=True)
        generated_ids = generated_ids.detach().cpu().numpy()

        preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

        predsictions.append(preds[0])
        truths.append(item["target"])

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
compute_metrics(predsictions, truths)

In [79]:
i = np.random.randint(len(val_data))

input_ids = tokenizer.encode(val_data[i]["input"], return_tensors="pt", add_special_tokens=True)
generated_ids = model.generate(input_ids=input_ids, max_length=512, repetition_penalty=2.5, early_stopping=True)
preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

n = '\n'
print(f"Preds: {preds[0]}\nTruth: {val_data[i]['target'].replace(n, ' ')}")

Preds: Требования: Опыт работы на аналогичной должности не менее 3 лет — Удостоверение/ запись в трудовой книжке от 4 разряда - Отсутствие ограничений по здоровью, согласно специальности Условия Рабочее место (г.Чебоксары) – Повременно-премиальная заработная платa за руки и медосмотр обеспечивается За счет работодателя Обязанности следующие
Truth: Требования: Опыт работы на аналогичной должности не менее 3 лет — Удостоверение/ запись в трудовой книжке от 4 разряда — Отсутствие ограничений по здоровью в соответствии со специальностью Условия: Рабочее место на территории завода промышленного литья (г.Чебоксары, Чувашская Республика) — Повременно-премиальная &quot;белая&quot; оплата труда от 90 000 руб/мес на руки за дневные смены, работа в ночное время с доплатой 40% — Своевременная выплата заработной платы, БЕЗ задержек — График работы 6/1 по 11 часов — Официальное трудоустройство согласно ТК РФ — Спецодежда и СИЗ предоставляются, — На заводе имеется столовая и оборудованные места для п

In [ ]:
# def make_json_dataset(df):
#     json_dataset = []

#     for _, row in df.iterrows():
#         part = {'context': row['full_text']}

#         if row['reqr'] != '':
#             part['qas'] = [
#                 {
#                     "id": row['id'] + "0",
#                     "is_impossible": False,

#                     "question": 'Каковы требования к соискателю?',
#                     "answers": [
#                         {
#                             "text": row['reqr'],
#                             "answer_start": row['full_text'].find(row['reqr']),
#                         }
#                     ],
#                 }
#             ]
#         else:
#             part['qas'] = [
#                 {
#                     "id": row['id'] + "1",
#                     "is_impossible": True,

#                     "question": 'Каковы требования к соискателю?',
#                     "answers": [],
#                 }
#             ]

#         if row['terms'] != '':
#             part['qas'].append(
#                 {
#                     "id": row['id'] + "0",
#                     "is_impossible": False,

#                     "question": 'Каковы условия работы?',
#                     "answers": [
#                         {
#                             "text": row['terms'],
#                             "answer_start": row['full_text'].find(row['terms']),
#                         }
#                     ],
#                 }
#             )
#         else:
#             part['qas'].append(
#                 {
#                     "id": row['id'] + "1",
#                     "is_impossible": True,

#                     "question": 'Каковы условия работы?',
#                     "answers": [],
#                 }
#             )

#         json_dataset.append(part)

#     return json_dataset

In [ ]:
train_json = make_json_dataset(train_data)
# val_json = make_json_dataset(val_data)

In [ ]:
# train_json

In [ ]:
# import wandb

In [ ]:
# sweep_config = {"parameters": {"num_train_epochs": {"values": [2, 3, 5]}}}
# sweep_id = wandb.sweep(sweep_config, project="Simple Sweep")
# wandb.init()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [ ]:
model = QuestionAnsweringModel(
    'distilbert', 'abletobetable/distilbert-ru-qa', #sweep_config=wandb.config
)

In [ ]:
context_text = 'Слесарь-Ремонтник | Автослесарь универсал по ремонту грузовых авто\nТребования: рассматриваем кандидатов без опыта работы, опыт ремонта грузовых авто MERCEDES  приветствуется. Обязанности: Диагностика и ремонт тягачей и полуприцепов. Выполнение работы по установке, регулированию и замене запасных частей, агрегатов и оборудования согласно оформленного заказ-наряда. Устраняет выявленные в ходе диагностики дефекты и неисправности по согласованию с механиком.  Условия:  Обучение. Своевременная выплата зарплаты. Комфортные боксы для ремонта с отоплением. Комната приема пищи, душ.'

predictions, raw_outputs = model.predict(
    [
        {
            "context": context_text,
            "qas": [
                {
                    "question": 'Каковы требования к соискателю?',
                    "id": "0",
                }
            ],
        }
    ]
)


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 21620.12it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
predictions

In [ ]:
model.train_model(train_json, output_dir='outputs/')

wandb.join()

wandb.agent(sweep_id, train)

In [ ]:
result, model_outputs = model.eval_model(val_json)

add example index and unique id: 100%|██████████| 200/200 [00:00<00:00, 723779.81it/s]


Running Evaluation:   0%|          | 0/34 [00:00<?, ?it/s]

In [ ]:
result, model_outputs